In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, Flatten, Conv1D, MaxPool1D

In [4]:
def mlp(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        hidden_layers=[32, 16, 8],
        dropout=0.0):
    
    # Flatten 2d input into 1d
    model = Sequential()
    model.add(Flatten(input_shape=(input_shape[-2], input_shape[-1])))
    # Add Dense + Dropout for every hidden layer
    for hidden_units in hidden_layers:
        model.add(Dense(hidden_units))
        model.add(Dropout(dropout))
    # Output layer to predict {horizon} time steps ahead
    model.add(Dense(horizon))

    model.compile(optimizer=optimizer, loss=loss)

    return model

In [6]:
def cnn(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        conv_blocks=[[64, 7, 2], [128, 5, 2]],
        dense_layers=[],
        dense_dropout=0.0):
    
    conv_layers = [b[0] for b in conv_blocks]
    kernels = [b[1] for b in conv_blocks]
    pool_sizes = [b[2] for b in conv_blocks]
    
    model = Sequential()
    
    model.add(Conv1D(conv_layers[0], kernels[0], 
                     activation="relu", padding="same", 
                     input_shape=(input_shape[-2], input_shape[-1])))
    
    if pool_sizes[0]:
        model.add(MaxPool1D(pool_size=pool_sizes[0]))
    
    for chanels, kernel, pool_size in zip(conv_layers[1:], kernels[1:], pool_sizes[1:]):
        model.add(Conv1D(chanels, kernel, activation="relu", padding="same"))
        if pool_size:
            model.add(MaxPool1D(pool_size=pool_size))
    # Dense block
    model.add(Flatten())
    for hidden_units in dense_layers:
        model.add(Dense(hidden_units))
        model.add(Dropout(dense_dropout))
    model.add(Dense(horizon))

    model.compile(optimizer=optimizer, loss=loss)
    
    return model

In [7]:
def lstm(input_shape,
        horizon=1,
        optimizer="adam",
        loss="mae",
        lstm_layers=[50],
        lstm_dropout=0.0,
        return_sequences=False,
        dense_layers=[],
        dense_dropout=0.0):
    
    model = Sequential()
    
    return_seq = return_sequences if len(lstm_layers) == 1 else True
    model.add(LSTM(lstm_layers[0], return_sequences=return_seq, 
                   dropout=lstm_dropout, input_shape=(input_shape[-2], input_shape[-1])))
    
    for i, u in enumerate(lstm_layers[1:]):
        return_seq = return_sequences if i == len(lstm_layers) - 2 else True
        
        model.add(LSTM(u, return_sequences=return_seq, dropout=lstm_dropout))
    
    if return_sequences:
        model.add(Flatten())
    
    for units in dense_layers:
        model.add(Dense(units))
        model.add(Dropout(dense_dropout))
    
    model.add(Dense(horizon))
    
    model.compile(optimizer=optimizer, loss=loss)
    
    return model

In [8]:
model = lstm((10,3,3), lstm_layers=[50,25,10], horizon=5, lstm_dropout=0.2, dense_layers=[100, 25], dense_dropout=0.2)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 50)             10800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 25)             7600      
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                1440      
_________________________________________________________________
dense (Dense)                (None, 100)               1100      
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 25)                2525      
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0